Set up environment

In [1]:
!pip install tensorflow
!pip install pandas
!pip install scikit-learn
import numpy as np
import pandas as pd
import tensorflow as tf
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Prepare the initial training data

In [2]:
# Load Data
train_df = pd.read_csv("TrainingDataBinary.csv")
test_df = pd.read_csv("TestingDataBinary.csv")

# Split features and target
x_train, x_test, y_train, y_test = train_test_split(train_df.iloc[:, :-1].values, train_df.iloc[:, -1].values, test_size=0.15)

# Preprocessing - Scaling the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

Set up the models

In [4]:
layerTypes = {
    "dense" : lambda x : Dense(x),
    "relu" : Activation("relu"),
    "sigmoid" : Activation("sigmoid"),
    "tanh" : Activation("tanh"),
    "softmax" : Activation("softmax"),
    "dropout" : lambda x : Dropout(x),

}

# List of available optimizers
optimizers = ["sgd", "adam", "rmsprop", "adagrad", "adadelta", "nadam", "ftrl"]

activators = ["relu", "sigmoid","tanh", "softmax"]

# List of available loss functions
loss_functions = [
    tf.keras.losses.MeanSquaredError(),
    tf.keras.losses.MeanAbsoluteError(),
    tf.keras.losses.BinaryCrossentropy(),
    tf.keras.losses.CategoricalCrossentropy(),
    tf.keras.losses.SparseCategoricalCrossentropy(),
    tf.keras.losses.Hinge(),
    tf.keras.losses.SquaredHinge(),
    tf.keras.losses.LogCosh(),
    tf.keras.losses.KLDivergence(),
    tf.keras.losses.Poisson(),
    tf.keras.losses.CosineSimilarity(),
    tf.keras.losses.Huber()
]

class Model:

  #Randomly create a dense (and potentially activation/dropout) layers
  def addLayer(self, start):
    layer = []
    nodes = 2 ** random.randint(3,8)
    if start:
      self.model.add(Dense(max(nodes,32), input_shape=(X_train_scaled.shape[1],)) )
    else:
      self.model.add(Dense(nodes))
    layer.append(("dense",nodes))
    if random.random() > 0.4:
      active = random.choice(activators)
      self.model.add(Activation(active))
      layer.append((active,0))

    if (dropout := random.random()) > 0.8:
      self.model.add(Dropout(dropout - 0.8))
      layer.append(("dropout", dropout - 0.8))
    self.layers.append(layer)

  def run(self):
    print(self.layers)
    self.model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_split=0.2)
    print(res := self.model.evaluate(X_test_scaled, y_test))
    self.accuracy = res[1]

class NewModel(Model):
  #Create a model from scratch
  def __init__(self, layercount = 3):
    while True:
      try:
        self.model = Sequential()
        self.layers = []
        for i in range(layercount):
          self.addLayer(i == 0)
        self.model.add(Dense(1))

        self.model.add(Activation(activator := random.choice(activators)))
        layer = [("dense", 1),(activator,0)]
        self.layers.append(layer)
        lastLayer = ((random.choice(optimizers),random.choice(loss_functions)))
        self.layers.append(lastLayer)

        self.model.compile(optimizer=lastLayer[0], loss=lastLayer[1], metrics=['accuracy'])
        self.run()
        break
      except Exception as e:
        print(e)


class EvolvedModel(Model):

  #Randomly merge 2 models together
  def __init__(self, inputLayers, inputLayers2):
    while True:
      try:
        self.model = Sequential()
        self.layers = []
        for i in range(min(len(inputLayers),len(inputLayers2))):
          if random.random() > 0.5:
            self.layers.append(inputLayers[i])
          else:
            self.layers.append(inputLayers2[i])
        first = True
        for layer in self.layers[:-1]:
          for i in layer:
            if i[0] in activators:
              self.model.add(layerTypes[i[0]])
            elif i[0] in ["dense", "dropout"]:
              if first:
                self.model.add(Dense(max(i[1],32), input_shape=(X_train_scaled.shape[1],)))
              else:
                self.model.add(layerTypes[i[0]](i[1]))

            else:
              print(f"Error: layer not recognised: {i[0]}")
        print(self.layers)

        self.model.compile(optimizer=self.layers[-1][0], loss=self.layers[-1][1], metrics=['accuracy'])
        self.run()
        break
      except Exception as e:
        print(e)


Evolutionarily find best algorithm

In [6]:

#Create the original models
AIs = [NewModel() for i in range(8)]
counter = 0
while True:
  #Sort the models to find the best
  AIs.sort(key=lambda AI: AI.accuracy, reverse=True)
  print(f"Round {counter :+= 1}")
  [print(f"{i}: {AIs[i].accuracy}: {AIs[i].layers}") for i in range(len(AIs))]
  newAIs = [AIs[0]]
  #Evolve the models
  for i in range(4):
    newAIs.append(EvolvedModel(AIs[0].layers,AIs[1].layers))
  #Introduce new models
  for i in range(3):
    newAIs.append(NewModel())
  AIs = newAIs

[[('dense', 32), ('tanh', 0)], [('dense', 32), ('dropout', 0.19972194437552548)], [('dense', 64), ('sigmoid', 0)], [('dense', 1), ('softmax', 0)], ('sgd', <keras.src.losses.losses.CategoricalCrossentropy object at 0x00000252A1346480>)]
Epoch 1/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5047 - loss: 0.0000e+00 - val_accuracy: 0.4775 - val_loss: 0.0000e+00
Epoch 2/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - accuracy: 0.5050 - loss: 0.0000e+00 - val_accuracy: 0.4775 - val_loss: 0.0000e+00
Epoch 3/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 532us/step - accuracy: 0.5148 - loss: 0.0000e+00 - val_accuracy: 0.4775 - val_loss: 0.0000e+00
Epoch 4/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step - accuracy: 0.5135 - loss: 0.0000e+00 - val_accuracy: 0.4775 - val_loss: 0.0000e+00
Epoch 5/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step - accuracy: 0.4895 - loss: 0.0000e+00 - val_accuracy: 0.4775 - val_loss: 0.0000e+00
Epoch 6/20
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 557us/step - accuracy: 0.5221 - l

NameError: name 'e' is not defined

In [ ]:
#[[('dense', 256)], [('dense', 32), ('relu', 0)], [('dense', 64), ('sigmoid', 0)], [('dense', 1), ('tanh', 0)], ('nadam', <keras.src.losses.BinaryCrossentropy object at 0x7e2e8a503a00>)]
#[[('dense', 8), ('relu', 0)], [('dense', 1), ('dropout', 0.05065572180245603)], [('dense', 32), ('tanh', 0)], [('dense', 1), ('tanh', 0)], ('rmsprop', <keras.src.losses.LogCosh object at 0x7800c126a1d0>)]
#[[('dense', 2)], [('dense', 32), ('tanh', 0), ('dropout', 0.010379300249843015)], [('dense', 128)], [('dense', 1), ('relu', 0)], ('sgd', <keras.src.losses.CategoricalCrossentropy object at 0x7800b995f2e0>)]

model.evaluate(X_test_scaled, y_test)[0]

Use the model to make predictions

In [ ]:
# Make predictions on the test set
predictions = model.predict(X_test_scaled)
predicted_labels = (predictions > 0.5).astype(int).flatten()

# Save predictions to CSV
output_df = pd.DataFrame(predicted_labels, columns=['PredictedLabel'])
output_df.to_csv('TestingResultsBinary.csv', index=False)

print("Model training complete and predictions saved.")
